In [3]:
import cv2
import numpy as np
import math

RGB_to_LMS = np.array([[0.3811, 0.5783, 0.0402],
                        [0.1967, 0.7244, 0.0782],
                        [0.0241, 0.1288, 0.8444]], dtype=np.float32)

i3 = 1 / math.sqrt(3)
i6 = 1 / math.sqrt(6)
i2 = 1 / math.sqrt(2)
LMS_to_lab = np.array([[i3, i3, i3],
                        [i6, i6, -2 * i6],
                        [i2, -i2, 0]], dtype=np.float32)

In [2]:
def convert_to_lab(input_image):
    img_rgb = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB).astype(np.float32)
    img_lms = cv2.transform(img_rgb, RGB_to_LMS)
    epsilon = 1.0 / 255
    min_mat = np.full_like(img_lms, epsilon)
    img_lms = np.maximum(img_lms, min_mat)
    img_lms = np.log10(img_lms) / np.log10(10)
    img_lab = cv2.transform(img_lms, LMS_to_lab)
    return img_lab

input_image = cv2.imread('palletimage.jpeg')
output_image_lab = convert_to_lab(input_image)

#cv2.imwrite("processed_convert_to_lab.jpeg", output_image_lab)
#cv2.imshow('Output Lab Image', output_image_lab)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np
import math

def convert_from_lab(input_image):
    img_lms = cv2.transform(input_image, LMS_to_lab)
    input_image = input_image.astype(np.float32)  # Convert to float32 if needed
    img_lms = cv2.transform(input_image, LMS_to_lab)
    img_lms_exp = cv2.pow(cv2.exp(img_lms), np.float32(math.log(10.0)))
    
    temp = cv2.invert(RGB_to_LMS)[1]
    img_rgb = cv2.transform(img_lms_exp, temp)
    
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    
    return img_bgr

# Example usage
input_image = cv2.imread('palletimage.jpeg', cv2.IMREAD_COLOR)
output_image_bgr = convert_from_lab(input_image)

#cv2.imshow('Output BGR Image', output_image_bgr)
#cv2.waitKey(0)
#cv2.destroyAllWindows()


In [6]:
def resize_image(image, new_width, new_height):
    return cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)

def final_adjustment(targetf, savedtf, TintVal, ModifiedVal):
    # Resize the saved image to match the dimensions of the target image
    savedtf = resize_image(savedtf, targetf.shape[1], targetf.shape[0])

    # If TintVal is not 1.0, compute a weighted average of the processed image and its grayscale representation.
    if TintVal != 1.0:
        grey = cv2.cvtColor(targetf, cv2.COLOR_BGR2GRAY)
        BGR = list(cv2.split(targetf))  # Convert tuple to list
        BGR[0] = TintVal * BGR[0] + (1.0 - TintVal) * grey
        BGR[1] = TintVal * BGR[1] + (1.0 - TintVal) * grey
        BGR[2] = TintVal * BGR[2] + (1.0 - TintVal) * grey
        targetf = cv2.merge(BGR)

    # If ModifiedVal is not 1.0, compute a weighted average of the processed image and the original target image.
    if ModifiedVal != 1.0:
        targetf = ModifiedVal * targetf + (1.0 - ModifiedVal) * savedtf

    return targetf

In [7]:
target_image = cv2.imread('palletimage.jpeg')
saved_image = cv2.imread('targetimage.jpeg')
TintVal = 0.5  # Example tint value (0.0 to 1.0)
ModifiedVal = 0.8  # Example modified value (0.0 to 1.0)

adjusted_image = final_adjustment(target_image, saved_image, TintVal, ModifiedVal)

# Display the adjusted image
cv2.imshow('Adjusted Image', adjusted_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
# This C++ code defines a function FullShading 
# that is used to adjust the shading of an image (targetf) based on a notional shader image (sourcef).
def FullShading(targetf, savedtf, sourcef, ExtraShading, ShaderVal):
    if ExtraShading:
        greyt = cv2.cvtColor(savedtf, cv2.COLOR_BGR2GRAY)
        greyp = cv2.cvtColor(targetf, cv2.COLOR_BGR2GRAY)
        greys = sourcef.copy()

        sdev = np.std(greys)
        smean = np.mean(greys)
        tdev = np.std(greyt)
        tmean = np.mean(greyt)

        greyt = (greyt - tmean) / tdev
        greyt = greyt * (ShaderVal * sdev + (1.0 - ShaderVal) * tdev) + ShaderVal * smean + (1.0 - ShaderVal) * tmean

        greyp = np.maximum(greyp, 1 / 255.0)

        chans = list(cv2.split(targetf))
        chans[0] = np.divide(chans[0], greyp)
        chans[0] = chans[0] * greyt
        chans[1] = np.divide(chans[1], greyp)
        chans[1] = chans[1] * greyt
        chans[2] = np.divide(chans[2], greyp)
        chans[2] = chans[2] * greyt

        targetf = cv2.merge(chans)

    return targetf

In [7]:
target_image = cv2.imread('targetimage.jpeg')
saved_image = cv2.imread('targetimage.jpeg')
source_image = cv2.imread('palletimage.jpeg')

# Apply FullShading with ExtraShading=True and ShaderVal=0.5
output_image = FullShading(target_image, saved_image, source_image, True, 0.5)

# Display the output image
cv2.imshow('Output Image', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
def SaturationProcessing(targetf, savedtf, SatVal):
    # Convert images to HSV color space
    targetf_hsv = cv2.cvtColor(targetf, cv2.COLOR_BGR2HSV)
    savedtf_hsv = cv2.cvtColor(savedtf, cv2.COLOR_BGR2HSV)

    if SatVal != 1:
        # Extract saturation channel from HSV images
        target_saturation = targetf_hsv[:, :, 1]
        saved_saturation = savedtf_hsv[:, :, 1]

        if SatVal < 0:
            # Compute SatVal as the ratio of largest saturation values
            SatVal = np.max(saved_saturation) / np.max(target_saturation)

        # Compute a weighted mix of target and saved saturation channels
        tmp_saturation = cv2.addWeighted(target_saturation, SatVal, saved_saturation, 1 - SatVal, 0.0)

        # Create a mask based on saturation difference
        mask = (target_saturation - tmp_saturation) > 0
        mask = mask.astype(np.uint8)

        # Create modified saturation channel based on mask
        modified_saturation = np.zeros_like(target_saturation, dtype=np.float32)
        modified_saturation = cv2.add(modified_saturation, tmp_saturation, dst=modified_saturation, mask=mask)
        modified_saturation = cv2.add(modified_saturation, target_saturation, dst=modified_saturation, mask=~mask)

        # Update the saturation channel in the HSV image
        targetf_hsv[:, :, 1] = modified_saturation

        # Convert back to BGR color space
        targetf = cv2.cvtColor(targetf_hsv, cv2.COLOR_HSV2BGR)

    return targetf